In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
car_sales_missing_data=pd.read_csv("car-sales-extended-missing-data.csv")

In [5]:
car_sales_missing_data

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


In [6]:
car_sales_missing_data.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [7]:
car_sales_missing_data.dropna(subset=['Price'], inplace=True)

In [8]:
car_sales_missing_data.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [11]:
car_sales_missing_data.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

In [10]:
#Split data into feature & label (i.e: X & y)
X=car_sales_missing_data.drop('Price',axis=1)
y=car_sales_missing_data['Price']

In [12]:
#Filling Null values

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

#categorical with 'missing' and numerical with mean
cat_impute=SimpleImputer(strategy='constant', fill_value='missing')
door_impute=SimpleImputer(strategy='constant', fill_value=4)
num_impute=SimpleImputer(strategy="mean")

#Define categorical and Numerical columns
cat_features=["Make","Colour"]
door_features=["Doors"]
num_features=["Odometer (KM)"]

#create an imputer (to fill the missing data)
imputer=ColumnTransformer([
    ('cat_impute',cat_impute, cat_features),
    ('door_impute',door_impute,door_features),
    ('num_impute',num_impute,num_features)
])

#Fit & transform the data
filled_X=imputer.fit_transform(X)

In [13]:
filled_X

array([['Honda', 'White', 4.0, 35431.0],
       ['BMW', 'Blue', 5.0, 192714.0],
       ['Honda', 'White', 4.0, 84714.0],
       ...,
       ['Nissan', 'Blue', 4.0, 66604.0],
       ['Honda', 'White', 4.0, 215883.0],
       ['Toyota', 'Blue', 4.0, 248360.0]], dtype=object)

In [14]:
filled_X.isna().sum()

AttributeError: 'numpy.ndarray' object has no attribute 'isna'

In [18]:
car_sales_filled=pd.DataFrame(filled_X, columns=['Make','Colour','Odometer (KM)','Doors'])

In [19]:
car_sales_filled

,Make,Colour,Odometer (KM),Doors
0,Honda,White,4,35431
1,BMW,Blue,5,192714
2,Honda,White,4,84714
3,Toyota,White,4,154365
4,Nissan,Blue,3,181577
...,...,...,...,...
945,Toyota,Black,4,35820
946,missing,White,3,155144
947,Nissan,Blue,4,66604
948,Honda,White,4,215883


In [20]:
car_sales_filled.head()

,Make,Colour,Odometer (KM),Doors
0,Honda,White,4,35431
1,BMW,Blue,5,192714
2,Honda,White,4,84714
3,Toyota,White,4,154365
4,Nissan,Blue,3,181577


In [21]:
car_sales_filled.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
dtype: int64

In [22]:
len(car_sales_filled)

950

In [23]:
len(y)

950

In [24]:
y.isna().sum()

0

In [25]:
#using one-Hot Encoding to convert categorical into Numerical

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

one_hot=OneHotEncoder()
categorical_columns=['Make','Colour','Doors']
transformer=ColumnTransformer([
    ('one_hot',one_hot, categorical_columns)], remainder="passthrough")

transformed_X=transformer.fit_transform(car_sales_filled)

In [26]:
transformed_X

<950x913 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [38]:
#applying random forest regression as supervised learning with continous variable

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test=train_test_split(transformed_X,y,test_size=0.2)


model=RandomForestRegressor()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.015619033719901587